# 202108090846_XGBoostRegressor_tree_sweep
Another sweep -- this time with some different hyperparameters, based on the [first sweep's results](https://wandb.ai/hushifang/202108_Kaggle_tabular_playground/sweeps/4abje3y9?workspace=user-hushifang). This one will be for tree models only (plus DART).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold#, KFold
from sklearn.metrics import mean_squared_error
import wandb
from wandb.xgboost import wandb_callback
# import timm
from pathlib import Path
import os
import math

In [2]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchinfo import summary
# # from fastai.callback.wandb import *
# from torch.cuda.amp import autocast, GradScaler

In [3]:
# from fastai.vision.all import *
# from fastai.tabular.all import *
# from fastai.callback.wandb import WandbCallback

In [4]:
# import xgboost as xgb
from xgboost import XGBRegressor, XGBRFRegressor
# from xgboost import XGBClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier

In [5]:
# %config Completer.use_jedi = False

In [6]:
# set_seed(42, reproducible=True)

In [7]:
datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/202108_august/')

In [8]:
df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)

In [9]:
# df.isnull().sum().any() # False

In [10]:
# df.info(verbose=True, null_counts=True)

So no NaNs.

In [11]:
# df.describe()

In [12]:
y = df.loss

In [13]:
features = [x for x in df.columns if x != 'loss']

In [14]:
X = df[features]

In [15]:
len(X)

250000

In [16]:
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
id,,,,,,,,,,,,,,,,,,,,,
0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,1.09101,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,1.28138,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,1.24210,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,1.09130,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,1.10151,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


In [17]:
len(y), len(y.value_counts())

(250000, 43)

So, it seems that we're actually looking at a classification problem here -- not a regression one.

# XGBRegressor

For the Sweep, you have to encapsulate your configs, your `wandb.init`, and everything else into a single function that will be called on each sweep iteration.

In [18]:
# # first tree sweep
# sweep_config = {
#     "method": "grid", # try grid or random
#     "metric": {
#       "name": "rmse",
#       "goal": "minimize"   
#     },
#     "parameters": {
#         "booster": {
#             "values": ["gbtree", "dart"]
#         },
#         "max_depth": {
#             "values": [3, 6, 9, 12]
#         },
#         "learning_rate": {
#             "values": [0.1, 0.2, 0.3]
#         },
#         "subsample": {
#             "values": [1, 0.5]
#         },
#         "n_estimators": {
#             "values": [50, 125, 200]
#         }
#     }
# }

In [37]:
# second tree sweep
sweep_config = {
    "method": "bayes", # try grid or random
    "metric": {
      "name": "rmse",
      "goal": "minimize"   
    },
    "parameters": {
        "max_depth": {
            "distribution": "int_uniform", 
            "min": 2,
            "max": 4,
        },
        "learning_rate": {
            "distribution": "normal", 
            "mu": 0.2,
            "sigma": 0.1
            #             "values": [0.1, 0.2, 0.3]
        },
        "n_estimators": {
            "distribution": "int_uniform",
            "min": 175,
            "max": 400,
#             "values": [50, 125, 200]
        }
    }
}

In [38]:
# %env "WANDB_NOTEBOOK_NAME" "202108090846_XGBoostRegressor_tree_sweep"
os.environ['WANDB_NOTEBOOK_NAME'] = '20210810_XGBRegressor_tree_sweep2.ipynb'

In [39]:
sweep_id = wandb.sweep(sweep_config, project="202108_Kaggle_tabular_playground")

Create sweep with ID: b2zv3fsy
Sweep URL: https://wandb.ai/hushifang/202108_Kaggle_tabular_playground/sweeps/b2zv3fsy


In [40]:
def train():
    config_defaults = {
        "library": "xgboost",
        "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
        "booster": 'dart',
        "n_estimators": 200,
        "max_depth": 3,
        "learning_rate": 0.1,
        "test_size": 0.2,
    }

    wandb.init(
#         project="202108_Kaggle_tabular_playground",
        save_code=True,
        tags=['XGBoost'],
        name='20210810_XGBRegressor_tree_sweep2',
        notes='XGBRegressor sweep using dart trees, second sweep, using Bayes strategy with normal distributions on max_depth, learning_rate, and n_estimators',
        config=config_defaults)
    
    config = wandb.config
        
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=config.test_size, random_state=42)
    
    model = XGBRegressor(
        tree_method=config.tree_method,
        booster=config.booster,
        n_estimators=config.n_estimators, 
        max_depth=config.max_depth,
        learning_rate=config.learning_rate, 
        test_size=config.test_size,
        subsample=1,
        random_state=42,
        n_jobs=-1, 
        verbosity=2, 
    )
    wandb.log({'params': model.get_params()})
    model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
    y_preds = model.predict(X_valid)
    mse = mean_squared_error(y_valid, y_preds)
    rmse = math.sqrt(abs(mse))
    wandb.log({'mse':mse, 'rmse':rmse})
    print(f"MSE is {mse}\nRMSE is {rmse}")   
    

In [ ]:
# ?wandb.agent

In [41]:
wandb.agent(sweep_id, function=train)  

wandb: Agent Starting Run: k6834h2c with config:
wandb: 	learning_rate: 0.3297350667538069
wandb: 	max_depth: 4
wandb: 	n_estimators: 396
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)
/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


MSE is 62.60081204530977
RMSE is 7.91206749499205


_runtime,807
_timestamp,1628635133
_step,397
mse,62.60081
rmse,7.91207


_runtime,▁█
_timestamp,▁█
_step,▁█
mse,▁
rmse,▁


...

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)
/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


MSE is 62.134793253705084
RMSE is 7.882562607027304


_runtime,448
_timestamp,1628653647
_step,294
mse,62.13479
rmse,7.88256


_runtime,▁█
_timestamp,▁█
_step,▁█
mse,▁
rmse,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# (old)

In [25]:
# model = XGBRegressor(
#         tree_method='auto',
#         booster='dart',
#         n_estimators=200, 
#         max_depth=3,
#         learning_rate=0.2, 
#         subsample=1,
#         random_state=42,
# #         test_size=config.test_size,
#         n_jobs=-1, 
#         verbosity=1, 
#     )

In [26]:
# model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
# model.fit(X, y)

XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='auto', validate_parameters=1, verbosity=1)

In [32]:
# wandb.sklearn.plot_regressor(model, X_train, X_valid, y_train, y_valid,  model_name='RandomForestRegressor', )

In [28]:
# y_pred = model.predict(X)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [29]:
# y_pred[:10]

array([7.848101 , 8.531089 , 5.067412 , 7.5927815, 6.710878 , 7.399656 ,
       7.6240873, 7.5142703, 5.317679 , 6.242065 ], dtype=float32)

In [34]:
# from sklearn.model_selection import cross_val_score

In [35]:
# scores = cross_val_score(model, X, y, cv=5, n_jobs=-1)

In [36]:
# model.get_depth()

In [38]:
# y_valid_preds = model.predict(X_valid)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [39]:
# mse = mean_squared_error(y_valid, y_valid_preds)

In [40]:
# mse

61.500556614829804

In [41]:
# rmse = math.sqrt(mse)
# rmse

7.842229059064126

In [ ]:
# wandb.log({'rmse': rmse, 'mse': mse, 'y_valid_preds': y_valid_preds})

In [42]:
# rmses = [math.sqrt(abs(score)) for score in scores]
# rmses

# Inference

Let's give that a submit...

In [42]:
model = XGBRegressor(
        tree_method='auto',
        booster='dart',
        n_estimators=400, 
        max_depth=3,
        learning_rate=0.1522, 
        subsample=1,
        random_state=42,
#         test_size=config.test_size,
        n_jobs=-1, 
        verbosity=1, 
    )

In [43]:
# model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1522, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=400, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='auto', validate_parameters=1, verbosity=1)

In [32]:
# wandb.sklearn.plot_regressor(model, X_train, X_valid, y_train, y_valid,  model_name='RandomForestRegressor', )

A quick check.

In [44]:
y_pred = model.predict(X)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [45]:
y_pred[:10]

array([7.5242977, 8.158392 , 4.832366 , 7.419939 , 6.553831 , 7.712717 ,
       7.8510847, 7.165042 , 5.162752 , 5.5137396], dtype=float32)

In [46]:
test_df = pd.read_csv(datapath/'test.csv', index_col='id', low_memory=False)

In [47]:
test_df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
id,,,,,,,,,,,,,,,,,,,,,
250000,0.812665,15,-1.239120,-0.893251,295.5770,15.87120,23.04360,0.942256,29.898000,1.11394,...,0.446389,-422.332,-1.44630,1.69075,1.059300,-3.010570,1.94664,0.529470,1.386950,8.78767
250001,0.190344,131,-0.501361,0.801921,64.8866,3.09703,344.80500,0.807194,38.421900,1.09695,...,0.377179,10352.200,21.06270,1.84351,0.251895,4.440570,1.90309,0.248534,0.863881,11.79390
250002,0.919671,19,-0.057382,0.901419,11961.2000,16.39650,273.24000,-0.003300,37.940000,1.15222,...,0.990140,3224.020,-2.25287,1.55100,-0.559157,17.838600,1.83385,0.931796,2.336870,9.05400
250003,0.860985,19,-0.549509,0.471799,7501.6000,2.80698,71.08170,0.792136,0.395235,1.20157,...,1.396880,9689.760,14.77150,1.41390,0.329272,0.802437,2.23251,0.893348,1.359470,4.84833
250004,0.313229,89,0.588509,0.167705,2931.2600,4.34986,1.57187,1.118300,7.754630,1.16807,...,0.862502,2693.350,44.18050,1.58020,-0.191021,26.253000,2.68238,0.361923,1.532800,3.70660


In [48]:
X_test = test_df[features] # this is just for naming consistency

In [49]:
y_test_preds = model.predict(X_test)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [50]:
sample_df = pd.read_csv(datapath/'sample_submission.csv')

In [51]:
sample_df.loc[:, 'loss'] = y_test_preds

In [52]:
sample_df.head()

,id,loss
0,250000,8.155357
1,250001,4.386151
2,250002,7.510742
3,250003,7.391403
4,250004,7.604273


In [53]:
sample_df.to_csv('202108062038_XGBoost.csv', index=False)

In [51]:
# wandb.finish()